# SAA-Aggression Dataset Inference

In [2]:
"""
Dataset      : Saag01
Epochs       : 80 (total)
Iterations   : 1361 (train), 341 (val), 190 (test)
Batch_size   : 48
Iters/Epoch  : 28

Arch         : BNInception
Modality     : ARP
""";

In [1]:
import os
import numpy as np
import pandas as pd

In [62]:
def extract_index(basename):
    index = basename.split('_')[-1]
    return index

# list all files' abspath in a directory
def abs_listdir(directory):
    directory = os.path.abspath(directory)
    return [
        os.path.join(directory, filename)
        for filename in os.listdir(directory)
    ]

# format indexing
def format_files_indexing(file_paths):
    formatted_filepaths = []
    for file_path in file_paths:
        # assuming digit is located at the end of file name
        basename, ext = os.path.splitext(file_path)
        index = extract_index(basename)
        if index.isdigit():
            formatted_filename = basename.replace(index, "{:05d}".format(int(index))) + ext

            os.rename(file_path, formatted_filename)
            formatted_filepaths.append(formatted_filename)

    return sorted(formatted_filepaths)

In [63]:
SCORES_DIR = 'Scores/'

SCORES_MAPDIR = {
    'SAAG01':{
        'ARP': {
            '3_segments': os.path.join('Scores/', 'saag01_bni_arp_seg_3_scores_split_0'),
            '7_segments': os.path.join('Scores/', 'saag01_bni_arp_seg_7_scores_split_0')
        },
        'Flow': {
            '3_segments': os.path.join('Scores/', 'saag01_bni_flow_seg_3_scores_split_0'),
            '7_segments': ''
        }
    }
}

## Approximated Rank Pooling

In [74]:
SAAG01_ARP_3seg_dir = SCORES_MAPDIR['SAAG01']['ARP']['3_segments']  # choose the one with 3 segments
SAAG01_ARP_7seg_dir = SCORES_MAPDIR['SAAG01']['ARP']['7_segments']  # choose the one with 7 segments

In [67]:
# Index formatting for scores' files
# this is for sorting consistency
saag01_arp_3seg_files = format_files_indexing(abs_listdir(SAAG01_ARP_3seg_dir))
saag01_arp_7seg_files = format_files_indexing(abs_listdir(SAAG01_ARP_7seg_dir))

saag01_arp_3seg_files[10:15]

['/home/sliu/Desktop/inference/Scores/saag01_bni_arp_seg_3_scores_split_0/saag01_bni_arp_seg_3_scores_split_0_00308.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_arp_seg_3_scores_split_0/saag01_bni_arp_seg_3_scores_split_0_00336.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_arp_seg_3_scores_split_0/saag01_bni_arp_seg_3_scores_split_0_00364.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_arp_seg_3_scores_split_0/saag01_bni_arp_seg_3_scores_split_0_00392.npy',
 '/home/sliu/Desktop/inference/Scores/saag01_bni_arp_seg_3_scores_split_0/saag01_bni_arp_seg_3_scores_split_0_00420.npy']

In [68]:
# Load labels
split_val_0_file = 'saag01_validation_split_0.txt'
label_df = pd.read_csv(split_val_0_file, 
                       delim_whitespace=True,
                       header=None,
                       names=['path', 'n_frames', 'label']
                      )

# drop last 5 items since we used droplast=True, 
# hence batchsize = 48, and valset = 341; then the remainder is 5 
labels = label_df['label'][:-5]

labels.shape

(336,)

In [69]:
# Try one numpy score from a file
saag01_score_sample = np.load(saag01_arp_3seg_files[-1])

pd.DataFrame(saag01_score_sample).head()  # logits for two scores (before softmax)

,0,1
0,-0.042839,0.002628
1,-0.299858,0.272666
2,3.966003,-3.968745
3,0.857336,-0.856593
4,-4.014513,3.995522


In [71]:
saag01_score_sample.shape

(336, 2)

### One-stream Evaluation

#### 1. Validation results

In [72]:
# Num_segments: 3
for scorefile in saag01_arp_3seg_files:
    itername, _ = os.path.splitext(scorefile)
    itername = extract_index(itername)
    
    print("Iter: %s" % itername)
    scores = np.load(scorefile)
    pred1 = np.argmax(scores, axis=1)
    acc = np.mean(pred1 == labels)
    print("ARP Acc: %f" % acc)

Iter: 00028
ARP Acc: 0.627976
Iter: 00056
ARP Acc: 0.794643
Iter: 00084
ARP Acc: 0.875000
Iter: 00112
ARP Acc: 0.613095
Iter: 00140
ARP Acc: 0.916667
Iter: 00168
ARP Acc: 0.854167
Iter: 00196
ARP Acc: 0.872024
Iter: 00224
ARP Acc: 0.892857
Iter: 00252
ARP Acc: 0.892857
Iter: 00280
ARP Acc: 0.898810
Iter: 00308
ARP Acc: 0.904762
Iter: 00336
ARP Acc: 0.916667
Iter: 00364
ARP Acc: 0.904762
Iter: 00392
ARP Acc: 0.869048
Iter: 00420
ARP Acc: 0.833333
Iter: 00448
ARP Acc: 0.776786
Iter: 00476
ARP Acc: 0.904762
Iter: 00504
ARP Acc: 0.913690
Iter: 00532
ARP Acc: 0.922619
Iter: 00560
ARP Acc: 0.943452
Iter: 00588
ARP Acc: 0.946429
Iter: 00616
ARP Acc: 0.940476
Iter: 00644
ARP Acc: 0.940476
Iter: 00672
ARP Acc: 0.952381
Iter: 00700
ARP Acc: 0.949405
Iter: 00728
ARP Acc: 0.943452
Iter: 00756
ARP Acc: 0.940476
Iter: 00784
ARP Acc: 0.949405
Iter: 00812
ARP Acc: 0.946429
Iter: 00840
ARP Acc: 0.922619
Iter: 00868
ARP Acc: 0.958333
Iter: 00896
ARP Acc: 0.946429
Iter: 00924
ARP Acc: 0.952381
Iter: 0095

In [73]:
# Num segments: 7
for scorefile in saag01_arp_7seg_files:
    itername, _ = os.path.splitext(scorefile)
    itername = extract_index(itername)
    
    print("Iter: %s" % itername)
    scores = np.load(scorefile)
    pred1 = np.argmax(scores, axis=1)
    acc = np.mean(pred1 == labels)
    print("ARP Acc: %f" % acc)

Iter: 00028
ARP Acc: 0.750000
Iter: 00056
ARP Acc: 0.851190
Iter: 00084
ARP Acc: 0.889881
Iter: 00112
ARP Acc: 0.866071
Iter: 00140
ARP Acc: 0.872024
Iter: 00168
ARP Acc: 0.898810
Iter: 00196
ARP Acc: 0.892857
Iter: 00224
ARP Acc: 0.922619
Iter: 00252
ARP Acc: 0.928571
Iter: 00280
ARP Acc: 0.925595
Iter: 00308
ARP Acc: 0.925595
Iter: 00336
ARP Acc: 0.919643
Iter: 00364
ARP Acc: 0.925595
Iter: 00392
ARP Acc: 0.931548
Iter: 00420
ARP Acc: 0.946429
Iter: 00448
ARP Acc: 0.934524
Iter: 00476
ARP Acc: 0.922619
Iter: 00504
ARP Acc: 0.919643
Iter: 00532
ARP Acc: 0.931548
Iter: 00560
ARP Acc: 0.943452
Iter: 00588
ARP Acc: 0.952381
Iter: 00616
ARP Acc: 0.952381
Iter: 00644
ARP Acc: 0.958333
Iter: 00672
ARP Acc: 0.955357
Iter: 00700
ARP Acc: 0.949405
Iter: 00728
ARP Acc: 0.946429
Iter: 00756
ARP Acc: 0.946429
Iter: 00784
ARP Acc: 0.949405
Iter: 00812
ARP Acc: 0.952381
Iter: 00840
ARP Acc: 0.946429
Iter: 00868
ARP Acc: 0.946429
Iter: 00896
ARP Acc: 0.952381
Iter: 00924
ARP Acc: 0.946429
Iter: 0095

#### 2. Test results

In [ ]:
# load test labels
split_test_file = 'saag01_test_split.txt'
test_labels_df = pd.read_csv(split_test_file, 
                       delim_whitespace=True,
                       header=None,
                       names=['path', 'n_frames', 'label']
                      )
test_labels = test_labels_df['label']

In [77]:
# Num_segments: 3
test_score_file = os.path.join(SAAG01_ARP_3seg_dir,
                               'saag01_bni_arp_seg_3_test_scores.npy')

In [78]:
test_labels.shape

(190,)

In [85]:
# Num_segments: 3
print("Test iterations")
scores = np.load(test_score_file)
pred1 = np.argmax(scores, axis=1)
acc = np.mean(pred1 == test_labels)
print("ARP Test Acc: %f" % acc)

Test iterations
ARP Test Acc: 0.957895


In [87]:
# Num_segments: 7
test_score_file = os.path.join(SAAG01_ARP_7seg_dir,
                               'saag01_bni_arp_seg_7_test_scores.npy')

In [88]:
# Num_segments: 7
print("Test iterations")
scores = np.load(test_score_file)
pred1 = np.argmax(scores, axis=1)
acc = np.mean(pred1 == test_labels)
print("ARP Test Acc: %f" % acc)

Test iterations
ARP Test Acc: 0.942105


In [90]:
inference_summary = {
    'Saa-Aggression': {
        'ARP (3 segments)': 0.957895,
        'ARP (7 segments)': 0.942105
    }
}

pd.DataFrame(inference_summary)

,Saa-Aggression
ARP (3 segments),0.957895
ARP (7 segments),0.942105
